## Load Libraries

In [2]:
import sqlite3
import pandas as pd

## # 2. Load Your Dataset

In [8]:
df = pd.read_csv("Cleaned_Data.csv")  # update file name as needed
df.head()

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal
0,Montreal,1976,Aquatics,Diving,3m springboard,"KÃ–HLER, Christa",Women,GDR,East Germany,W,Silver
1,Montreal,1976,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze
2,Montreal,1976,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold
3,Montreal,1976,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver
4,Montreal,1976,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze


In [4]:
import os
os.getcwd()

'C:\\Users\\Chinmaya'

## Create SQLite In-Memory DB

In [10]:
conn = sqlite3.connect(":memory:")
df.to_sql("summer_olympic_medals", conn, index=False, if_exists="replace")

## Helper function to run SQL and show results

In [12]:
def run(sql):
    return pd.read_sql_query(sql, conn)

##  4. Handling Missing Data

In [14]:
run("""
SELECT *
FROM summer_olympic_medals
WHERE City IS NULL OR Year IS NULL OR Sport IS NULL OR Discipline IS NULL
   OR Event IS NULL OR Athlete IS NULL OR Gender IS NULL
   OR Country_Code IS NULL OR Country IS NULL OR Event_gender IS NULL OR Medal IS NULL;
""")

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal


## 5. Identifying Duplicate Rows

In [15]:
run("""
SELECT 
    City, Year, Sport, Discipline, Event, Athlete, Gender,
    Country_Code, Country, Event_gender, Medal, COUNT(*) AS count
FROM summer_olympic_medals
GROUP BY City, Year, Sport, Discipline, Event, Athlete, Gender,
         Country_Code, Country, Event_gender, Medal
HAVING COUNT(*) > 1;
""")

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal,count


## 6. Gender Ratio in Winning Teams

In [16]:
run("""
SELECT 
    Gender,
    COUNT(Gender) AS Distribution,
    ROUND(COUNT(Gender) * 100.0 / (SELECT COUNT(*) FROM summer_olympic_medals), 2) AS Percentage
FROM summer_olympic_medals
GROUP BY Gender
ORDER BY Percentage DESC;
""")

,Gender,Distribution,Percentage
0,Men,9386,61.29
1,Women,5928,38.71


## 7. Gender Distribution by Medal

In [18]:
run("""
SELECT 
    Gender, Medal,
    COUNT(*) AS Distribution,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM summer_olympic_medals), 2) AS Percentage
FROM summer_olympic_medals
GROUP BY Gender, Medal
ORDER BY Percentage DESC;
""")

,Gender,Medal,Distribution,Percentage
0,Men,Bronze,3257,21.27
1,Men,Gold,3078,20.10
2,Men,Silver,3051,19.92
3,Women,Bronze,2001,13.07
4,Women,Silver,1965,12.83
5,Women,Gold,1962,12.81


## 8. Country with Maximum Medals

In [20]:
run("""
SELECT Country, COUNT(Medal) AS TotalMedals
FROM summer_olympic_medals
GROUP BY Country
ORDER BY TotalMedals DESC
LIMIT 1;
""")

,Country,TotalMedals
0,United States,1992


##  9. Country with Only 1 Medal (Minimum)

In [21]:
run("""
SELECT Country, COUNT(Medal) AS TotalMedals
FROM summer_olympic_medals
GROUP BY Country
HAVING COUNT(Medal) = 1;
""")

,Country,TotalMedals
0,Afghanistan,1
1,Barbados,1
2,Bermuda*,1
3,Burundi,1
4,Cote d'Ivoire,1
5,Djibouti,1
6,Eritrea,1
7,Guyana,1
8,Kuwait,1
9,Lebanon,1


## 10. Distinct Events Count

In [22]:
run("""
SELECT Event, COUNT(Event) AS Distinct_Event_Count
FROM summer_olympic_medals
GROUP BY Event
ORDER BY Distinct_Event_Count DESC;
""")

,Event,Distinct_Event_Count
0,hockey,815
1,handball,780
2,football,669
3,volleyball,647
4,basketball,646
...,...,...
288,"59 - 64kg, total (featherweight)",3
289,56kg,3
290,"54 - 59kg, total (bantamweight)",3
291,"- 54kg, total (flyweight)",3


## 11. Olympic Host City Year-wise

In [23]:
run("""
SELECT City, Year, COUNT(City) AS Max_Hostings
FROM summer_olympic_medals
GROUP BY City, Year
ORDER BY Year ASC;
""")

,City,Year,Max_Hostings
0,Montreal,1976,1305
1,Moscow,1980,1385
2,Los Angeles,1984,1459
3,Seoul,1988,1546
4,Barcelona,1992,1705
5,Atlanta,1996,1859
6,Sydney,2000,2015
7,Athens,2004,1998
8,Beijing,2008,2042


## 12. City Hosting the Most Events

In [24]:
run("""
SELECT City, COUNT(Event) AS Events_Count
FROM summer_olympic_medals
GROUP BY City
ORDER BY Events_Count DESC
LIMIT 1;
""")

,City,Events_Count
0,Beijing,2042


## 13. Understanding Sports → Discipline → Event Structure

In [25]:
run("""
SELECT DISTINCT Sport, Discipline, Event
FROM summer_olympic_medals;
""")


,Sport,Discipline,Event
0,Aquatics,Diving,3m springboard
1,Aquatics,Diving,10m platform
2,Aquatics,Swimming,4x100m freestyle relay
3,Aquatics,Swimming,400m freestyle
4,Aquatics,Swimming,1500m freestyle
...,...,...,...
329,Sailing,Sailing,Laser Radial - One Person Dinghy
330,Sailing,Sailing,Finn - Heavyweight Dinghy
331,Sailing,Sailing,Laser - One Person Dinghy
332,Sailing,Sailing,RS:X - Windsurfer


## 14. Athlete With the Most Medals

In [26]:
run("""
SELECT Athlete, COUNT(Medal) AS Medal_Count
FROM summer_olympic_medals
GROUP BY Athlete
ORDER BY Medal_Count DESC
LIMIT 1;
""")

,Athlete,Medal_Count
0,"PHELPS, Michael",16


## 15. Country With Most Medals By Year

In [27]:
run("""
SELECT Country, Medal_Count, Year
FROM
    (SELECT Country, COUNT(Medal) AS Medal_Count, Year
     FROM summer_olympic_medals
     GROUP BY Country, Year) AS MedalData
WHERE Medal_Count = (
        SELECT MAX(Medal_Count)
        FROM (
            SELECT Country, COUNT(Medal) AS Medal_Count, Year
            FROM summer_olympic_medals
            GROUP BY Country, Year
        ) AS YearlyData
        WHERE YearlyData.Year = MedalData.Year
)
ORDER BY Year;
""")

,Country,Medal_Count,Year
0,Soviet Union,285,1976
1,Soviet Union,442,1980
2,United States,333,1984
3,Soviet Union,294,1988
4,United States,224,1992
5,United States,260,1996
6,United States,248,2000
7,United States,264,2004
8,United States,315,2008


## 16. Dominating Country in Wrestling

In [28]:
run("""
SELECT Country, COUNT(Medal) AS Total_Medals
FROM summer_olympic_medals
WHERE Sport = 'Wrestling'
GROUP BY Country
ORDER BY Total_Medals DESC
LIMIT 1;
""")

,Country,Total_Medals
0,United States,57
